In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers , Model
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import (LSTM, Embedding, BatchNormalization,Dense, TimeDistributed,Input, Dropout, Bidirectional,Flatten, GlobalMaxPool1D)
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from tensorflow.keras import preprocessing as kprocessing
from tensorflow.keras import models, layers, optimizers

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE

In [2]:
import os
dirname = r"C:/Users/www.abcom.in/Desktop/Python Projects"
filename = os.path.join(dirname, 'Topic Modelling/test.csv')

In [3]:
data = pd.read_csv(filename)
data.head()

,text,sentiment
0,"My daughter liked it but I was aghast, that a ...",neg
1,I... No words. No words can describe this. I w...,neg
2,this film is basically a poor take on the old ...,neg
3,"This is a terrible movie, and I'm not even sur...",neg
4,First of all this movie is a piece of reality ...,pos


In [4]:
d = {'neg': 0, 'pos': 1}
data['sentiment_class'] = data['sentiment'].map(d).fillna(data['sentiment'])

In [5]:
X_tf_train = data["text"].iloc[:1000].tolist()
X_tf_test = data["text"].iloc[1000:1200].tolist()

In [6]:
y_tf_train = data["sentiment_class"].iloc[:1000].tolist()
y_tf_test = data["sentiment_class"].iloc[1000:1200].tolist()

In [7]:
X_tf_train = np.loadtxt('train_doc2text.txt')
X_tf_test = np.loadtxt('test_doc2text.txt')
y_train = np.loadtxt('y_train.txt')
y_test = np.loadtxt('y_test.txt')

In [89]:
def define_model(train):
    input1 = Input(shape=(train.shape[1],1)) #take the reshape last two values, see "data = np.reshape(data,(10,2,1))" which is "data/batch-size, row, column"
    lstm1 = Bidirectional(LSTM(units=32,dropout=0.3))(input1)
    dnn_hidden_layer1 = Dense(8, activation='relu')(lstm1)
    dnn_output = Dense(2, activation='softmax')(dnn_hidden_layer1)
    model = Model(inputs=[input1],outputs=[dnn_output])
    # compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # , binary_crossentropy
    model.summary()
    return model

In [90]:
X_tf_test.shape

(200, 100)

In [91]:
# Reshape the data into 3-D numpy array
data = np.reshape(X_tf_train,(1000,100,1)) #Here we have a total of 10 rows or records
print("data after reshape => ",data)
# Call the model
model = define_model(X_tf_train)
# fit the model
model.fit(data,y_train,epochs=7,batch_size=32,verbose=4)

data after reshape =>  [[[ 0.35534066]
  [-0.38857463]
  [-1.11197364]
  ...
  [ 0.79235607]
  [ 0.60311311]
  [-0.35596308]]

 [[ 0.1274412 ]
  [ 1.6244688 ]
  [-1.23828971]
  ...
  [ 0.37799245]
  [ 0.58831179]
  [-1.02719963]]

 [[-0.21048264]
  [ 0.26734981]
  [-0.36258751]
  ...
  [ 0.59145188]
  [-0.43736786]
  [ 0.04750437]]

 ...

 [[ 0.42633426]
  [ 1.59695601]
  [ 1.33299792]
  ...
  [-0.52192014]
  [ 1.24278474]
  [-0.08308743]]

 [[-0.39181429]
  [ 0.64391202]
  [-0.1328595 ]
  ...
  [-0.22893402]
  [ 0.01382251]
  [-0.1203945 ]]

 [[-0.04077477]
  [ 1.01108563]
  [ 0.20022759]
  ...
  [-0.59707397]
  [ 0.22835319]
  [-0.5700615 ]]]
Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 100, 1)]          0         
                                                                 
 bidirectional_17 (Bidirecti  (None, 64)               8704      

In [92]:
X_tf_test.shape

(200, 100)

In [93]:
test_data = X_tf_test
print(test_data.shape)
# reshape the test data
test_data = np.reshape(test_data,(200,100,1))
print(test_data)

(200, 100)
[[[-0.49800488]
  [-0.71064037]
  [ 0.44375345]
  ...
  [ 0.89091736]
  [-0.10834965]
  [-0.159852  ]]

 [[-0.64948803]
  [ 0.57907456]
  [ 0.45675436]
  ...
  [ 0.11758445]
  [-0.67187721]
  [ 0.60432458]]

 [[ 0.86537099]
  [-0.21732418]
  [ 0.12563829]
  ...
  [ 0.29958752]
  [ 1.10510325]
  [-0.53152889]]

 ...

 [[ 0.3662293 ]
  [ 1.00022769]
  [-0.13192451]
  ...
  [-0.18154313]
  [-0.78746539]
  [-1.20234346]]

 [[ 0.00604014]
  [ 0.04512979]
  [-0.39498132]
  ...
  [-0.29241902]
  [ 0.08400491]
  [-0.18855789]]

 [[ 0.60609537]
  [ 0.43715924]
  [-1.04382384]
  ...
  [-0.72153741]
  [ 0.54407531]
  [-0.4089798 ]]]


In [94]:
# Take a test data to test the working of the model
pred = model.predict(test_data)
print("predicted sigmoid output => ",pred)

7/7 [==============================] - 1s 10ms/step
predicted sigmoid output =>  [[0.51579803 0.4842019 ]
 [0.55453175 0.4454682 ]
 [0.5083439  0.49165612]
 [0.5477258  0.45227423]
 [0.5085834  0.4914165 ]
 [0.5267155  0.47328445]
 [0.51356786 0.48643214]
 [0.56123775 0.43876225]
 [0.5678264  0.43217358]
 [0.5101496  0.48985043]
 [0.54809386 0.4519062 ]
 [0.479052   0.52094793]
 [0.5463058  0.45369428]
 [0.52435786 0.4756421 ]
 [0.5760285  0.42397153]
 [0.4866428  0.5133572 ]
 [0.5125905  0.4874094 ]
 [0.5746937  0.4253063 ]
 [0.554658   0.44534194]
 [0.5641485  0.43585157]
 [0.53472185 0.4652782 ]
 [0.5375843  0.4624157 ]
 [0.5421003  0.4578997 ]
 [0.5626851  0.43731496]
 [0.50279    0.4972101 ]
 [0.51787037 0.4821297 ]
 [0.47857302 0.521427  ]
 [0.5312941  0.4687059 ]
 [0.57997704 0.42002293]
 [0.5519629  0.44803712]
 [0.5100994  0.4899006 ]
 [0.5100558  0.48994425]
 [0.54545856 0.45454144]
 [0.5499353  0.4500647 ]
 [0.5089286  0.49107137]
 [0.5297392  0.47026074]
 [0.5134348  0.4865